In [1]:
# Install and import the necessary libraries
# !pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [2]:
import os
import torch
from datasets import load_dataset
from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    pipeline,
    logging,
)

In [3]:
# Model
base_model = "microsoft/phi-2"
# new_model = "phi-2-medquad"

# # Dataset
# dataset = load_dataset("prsdm/MedQuad-phi2-1k", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [4]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
original_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map='auto',
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [24]:
from datasets import load_dataset

# Load SNLI dataset
snli = load_dataset("snli")

test_indices = list(range(0, 10000, 100))[:100]      # Get first 100 samples

test_samples = snli['test'].select(test_indices)

In [ ]:
c=0
base_results = []
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
for data in test_samples:
    premise = data['premise']
    hypothesis = data['hypothesis']
    true_label = data['label']
    # print(premise, hypothesis)
    prompt = (
    "Determine the relationship between the premise and hypothesis as one of the following: "
    "entailment, neutral, or contradiction.\n\n"
    "Example :\n"
    "Premise: 'A man inspects the uniform of a figure in some East Asian country.'\n"
    "Hypothesis: 'The man is inspecting his own uniform.'\n"
    "The relationship is: contradiction\n\n"
    # "Example 2:\n"
    # "Premise: 'A soccer game with multiple males playing.'\n"
    # "Hypothesis: 'Some men are playing a sport.'\n"
    # "The relationship is: entailment\n\n"
    "Now, classify the following:\n"
    f"Premise: '{premise}'\n"
    f"Hypothesis: '{hypothesis}'\n"
    "The relationship is:"
    )
    inputs = tokenizer(prompt, return_tensors="pt")

    output_ids = original_model.generate(**inputs, max_length=inputs['input_ids'].shape[1] + 100,
                                pad_token_id=tokenizer.eos_token_id)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # print(output_text)
    # print("*********************************")
    if "correct answer is" in output_text:
        extracted_str = output_text.split("correct answer is")[-1].strip()
        extracted_result = extracted_str.split("\n")[0]
    else:
        extracted_str = output_text.split("Now")[0].strip()
        extracted_str2 = extracted_str.split("The relationship is")[-1].strip()
        extracted_result = extracted_str2.split("\n")[0]
        if extracted_result.split()[0]=="Determine":
            extracted_result=extracted_result.split()[-1]

    if true_label==0 and ("entailment" in extracted_result or "Entailment" in extracted_result):
            c+=1
    elif true_label==1 and ("neutral" in extracted_result or "Neutral" in extracted_result):
            c+=1
    elif true_label==2 and ("contradiction" in extracted_result or "Contradiction" in extracted_result):
            c+=1
    # print(f"Premise: {premise}")
    # print(f"Hypothesis: {hypothesis}")
    # print(f"Predicted extracted relationship: {extracted_result}")
    base_results.append(extracted_result.split(',')[0])

In [ ]:
print(base_results)

['Neutral.', 'Entailment.', 'Entailment.', 'entailment', 'Entailment.', 'Neutral.', 'entailment', 'entailment', 'entailment', 'Entailment.', 'Entailment.', 'entailment', 'entailment', 'Neutral.', 'Entailment.', 'entailment', 'Entailment.', 'Entailment.', 'Entailment.', 'Entailment.', 'Entailment.', 'Neutral.', 'Entailment.', 'Entailment.', 'entailment', 'entailment', 'entailment', 'Contradiction.', 'entailment', 'Entailment.', 'Entailment.', 'entailment', 'entailment', 'Entailment.', 'Entailment.', 'entailment', 'Neutral.', 'Neutral.', 'Neutral.', 'entailment', 'entailment', 'Entailment.', 'Entailment.', 'Contradiction.', 'Neutral.', 'Contradiction.', 'Entailment.', 'Entailment.', 'Entailment.', 'Neutral.', 'Entailment.', 'Neutral.', 'Entailment.', 'Neutral.', 'Entailment.', 'Entailment.', 'Entailment.', 'entailment', 'entailment', 'entailment', 'Entailment.', 'Entailment.', 'entailment', 'Entailment.', 'Contradiction.', 'Neutral.', 'Entailment.', 'entailment', 'entailment', 'entailmen

In [27]:
acc = c/len(test_samples)
print("Accuracy of the base model :",acc)

Accuracy of the base model : 0.25
